In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
dataset_path = '/content/drive/MyDrive/keyword_extraction_ml_2022/preprocessed.csv'
data = pd.read_csv(dataset_path)

In [ ]:
!pip install datasets==1.0.2
!pip install transformers==4.2.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.2.1-py3-none-any.whl (1.8 MB)
  Using cached sacremoses-0.0.53-py3-none-any.whl
  Using cached tokenizers-0.9.4.tar.gz (184 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: Could not build wheels for tokenizers, which is required to install pyproject.toml-based projects


In [ ]:
import datasets
train_data = datasets.load_dataset("csv", data_files=dataset_path, split="train")

Using custom data configuration default
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-cac37c038c819445/0.0.0/0d06ce3712951dae7909fb214283b88efab3578535edb5eebd37c498b7a35277)


In [ ]:
print(train_data)

Dataset(features: {'text': Value(dtype='string', id=None), 'keywords': Value(dtype='string', id=None)}, num_rows: 137879)


In [ ]:
train_data.info

DatasetInfo(description='', citation='', homepage='', license='', features={'text': Value(dtype='string', id=None), 'keywords': Value(dtype='string', id=None)}, post_processed=None, supervised_keys=None, builder_name='csv', config_name='default-cac37c038c819445', version=0.0.0, splits={'train': SplitInfo(name='train', num_bytes=201096825, num_examples=137879, dataset_name='csv')}, download_checksums={'/content/drive/MyDrive/keyword_extraction_ml_2022/preprocessed.csv': {'num_bytes': 200407445, 'checksum': '5eabe11631e7a1bc87f67ab08280fb533ac3788b18194f0f9e51262ddc526e92'}}, download_size=200407445, post_processing_size=None, dataset_size=201096825, size_in_bytes=401504270)

In [ ]:
import pandas as pd
from IPython.display import display, HTML
from datasets import ClassLabel

df = pd.DataFrame(train_data[:1])
for column, typ in train_data.features.items():
      if isinstance(typ, ClassLabel):
          df[column] = df[column].transform(lambda i: typ.names[i])
display(HTML(df.to_html()))

,text,keywords
0,amaç dünya sağlık örgütü dsö gelişmiş ülkelerde nüfusun 10unu gelişmekte olan ülkelerde 12sini özürlülerin oluşturduğunu kabul etmektedir devlet istatistik enstitüsünün yapmış olduğu 2002 türkiye özürlüler araştırması sonuçlarına göre ülkemizde özürlü olan nüfusun toplam nüfus içindeki oranı 1229dur çalışmamızda özürlülerle ilgili ülkemiz verilerine katkıda bulunmak amacıyla yapılmıştır yöntemler hastanemiz özürlü kuruluna 2011 yılı ocak aralık aylarında başvuran 1805 vakaya dosyalar geriye dönük olarak değerlendirilmiştir bulgular 1805 vakanın 454 820 kişi kadın 546 sı 985 kişi erkektir başvuru 228 126 vaka mart ayında gerçekleşmiştir 50den özrü olduğuna karar verilen 1173 65 kişi varken kalan 541 30 kişinin özrünün 50 üzerinde olduğu tespit edilmiştir verilen özür oranlarının süreleri incelendiği zaman tüm başvurular içerisinde 873 olgunun 484 sürekli özre sahip olduğu görülmüştür sonuç özürlülüğe yol açan etmenler minimuma indirilmelidir alınacak tedbirler büyükşehir kent merkezinde değil diğer küçük yerleşim bölgelerinde uygulanmalıdır,özürlülük özür oranı diyarbakır sağlık kurulu


In [ ]:
!pip install transformers
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("dbmdz/bert-base-turkish-128k-uncased")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def map_to_length(x):
  x["text_len"] = len(tokenizer(x["text"]).input_ids)
  x["text_longer_512"] = int(x["text_len"] > 512)
  x["keywords_len"] = len(tokenizer(x["keywords"]).input_ids)
  x["keywords_longer_64"] = int(x["keywords_len"] > 64)
  x["keywords_longer_128"] = int(x["keywords_len"] > 128)
  return x

In [ ]:
!pip install dill==0.3.5.1
sample_size = 10000
data_stats = train_data.select(range(sample_size)).map(map_to_length, num_proc=4)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Token indices sequence length is longer than the specified maximum sequence length for this model (790 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (558 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (811 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1188 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
def compute_and_print_stats(x):
  if len(x["text_len"]) == sample_size:
    print(
        "Text Mean: {}, %-Texts > 512:{}, keywords Mean:{}, %-Keywords > 64:{}, %-Keywords > 128:{}".format(
            sum(x["text_len"]) / sample_size,
            sum(x["text_longer_512"]) / sample_size,
            sum(x["keywords_len"]) / sample_size,
            sum(x["keywords_longer_64"]) / sample_size,
            sum(x["keywords_longer_128"]) / sample_size,
        )
    )

output = data_stats.map(
  compute_and_print_stats,
  batched=True,
  batch_size=-1,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

Text Mean: 191.5502, %-Texts > 512:0.036, keywords Mean:11.4447, %-Keywords > 64:0.01, %-Keywords > 128:0.0082


In [ ]:
encoder_max_length=512
decoder_max_length=128

In [ ]:
def process_data_to_model_inputs(batch):
  # tokenize the inputs and labels
  inputs = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=encoder_max_length)
  outputs = tokenizer(batch["keywords"], padding="max_length", truncation=True, max_length=decoder_max_length)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  batch["labels"] = outputs.input_ids.copy()

  # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`.
  # We have to make sure that the PAD token is ignored
  batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return batch

In [ ]:
# train_data = train_data.select(range(32))

In [ ]:
batch_size = 16
# batch_size=4

train_data = train_data.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["text", "keywords"]
)

  0%|          | 0/8618 [00:00<?, ?ba/s]

In [ ]:
train_data

Dataset(features: {'attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}, num_rows: 137879)

In [ ]:
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"],
)

In [ ]:
val_data = datasets.load_dataset("csv", data_files=dataset_path, split="train[:10%]")

Using custom data configuration default
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-cac37c038c819445/0.0.0/0d06ce3712951dae7909fb214283b88efab3578535edb5eebd37c498b7a35277)


In [ ]:
val_data = val_data.select(range(8))

In [ ]:
val_data = val_data.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["text", "keywords"]
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"],
)

In [ ]:
from transformers import EncoderDecoderModel

bert2bert = EncoderDecoderModel.from_encoder_decoder_pretrained("dbmdz/bert-base-turkish-128k-uncased", "dbmdz/bert-base-turkish-128k-uncased")


Some weights of the model checkpoint at dbmdz/bert-base-turkish-128k-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at dbmdz/bert-base-turkish-128k-uncased were not used when initiali

In [ ]:
bert2bert

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(128000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, eleme

In [ ]:
bert2bert.save_pretrained("bert2bert")



In [ ]:
bert2bert = EncoderDecoderModel.from_pretrained("bert2bert")



In [ ]:
bert2bert.config

EncoderDecoderConfig {
  "_commit_hash": null,
  "_name_or_path": "bert2bert",
  "architectures": [
    "EncoderDecoderModel"
  ],
  "decoder": {
    "_name_or_path": "dbmdz/bert-base-turkish-128k-uncased",
    "add_cross_attention": true,
    "architectures": null,
    "attention_probs_dropout_prob": 0.1,
    "bad_words_ids": null,
    "begin_suppress_tokens": null,
    "bos_token_id": null,
    "chunk_size_feed_forward": 0,
    "classifier_dropout": null,
    "cross_attention_hidden_size": null,
    "decoder_start_token_id": null,
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": null,
    "exponential_decay_length_penalty": null,
    "finetuning_task": null,
    "forced_bos_token_id": null,
    "forced_eos_token_id": null,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 768,
    "id2label": {
      "0": "LABEL_0",
      "1": "LABEL_1"
    },
    "initializer_range

In [ ]:
bert2bert.config.decoder_start_token_id = tokenizer.cls_token_id
bert2bert.config.eos_token_id = tokenizer.sep_token_id
bert2bert.config.pad_token_id = tokenizer.pad_token_id
bert2bert.config.vocab_size = bert2bert.config.encoder.vocab_size

In [ ]:
bert2bert.config.max_length = 142
bert2bert.config.min_length = 4
bert2bert.config.no_repeat_ngram_size = 2
bert2bert.config.early_stopping = True
bert2bert.config.length_penalty = 2.0
bert2bert.config.num_beams = 4

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments



In [ ]:
!pip install git-python==1.0.3
!pip install rouge_score
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install transformers[torch]
!pip install accelerate -U
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    output_dir="./",
    # logging_steps=2,
    # save_steps=10,
    # eval_steps=4,
    logging_steps=1000,
    save_steps=500,
    eval_steps=7500,
    warmup_steps=2000,
    num_train_epochs=10,
    save_total_limit=3,
)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
rouge = datasets.load_metric("rouge")



In [ ]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [ ]:
trainer = Seq2SeqTrainer(
    model=bert2bert,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to p

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 9>:9                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1645 in train                    │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1938 in _inner_training_loop     │
│                                                                                                  │
│   1935 │   │   │   │   │   self.control = self.callback_handler.on_step_begin(args, self.state,  │
│   1936 │   │   │   │                                                                             │
│   1937 │   │   │   │   with self.accelerator.accumulate(model):                                  │
│ ❱ 1938 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1939 │   │   │   │                                                                             │
│   1940 │   │   │   │   if (                                                                      │
│   1941 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2759 in training_step            │
│                                                                                                  │
│   2756 │   │   │   return loss_mb.reduce_mean().detach().to(self.args.device)                    │
│   2757 │   │                                                                                     │
│   2758 │   │   with self.compute_loss_context_manager():                                         │
│ ❱ 2759 │   │   │   loss = self.compute_loss(model, inputs)                                       │
│   2760 │   │                                                                                     │
│   2761 │   │   if self.args.n_gpu > 1:                                                           │
│   2762 │   │   │   loss = loss.mean()  # mean() to average on multi-gpu parallel training        │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2784 in compute_loss             │
│                                                                                                  │
│   2781 │   │   │   labels = inputs.pop("labels")                                                 │
│   2782 │   │   else:                                                                             │
│   2783 │   │   │   labels = None                                                                 │
│ ❱ 2784 │   │   outputs = model(**inputs)                                                         │
│   2785 │   │   # Save past state if it exists              

In [ ]:
dummy_bert2bert = EncoderDecoderModel.from_pretrained("./checkpoint-20")



In [ ]:
from transformers import BertTokenizer

bert2bert = dummy_bert2bert.to("cuda")
tokenizer = BertTokenizer.from_pretrained("./checkpoint-20")

In [ ]:
test_data = datasets.load_dataset("csv", data_files=dataset_path, split="train[:1%]")



In [ ]:
def generate_summary(batch):
    # cut off at BERT max length 512
    inputs = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = bert2bert.generate(input_ids, attention_mask=attention_mask)

    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred_keywords"] = output_str

    return batch

In [ ]:
batch_size = 16  # change to 64 for full evaluation

results = test_data.select(range(10)).map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["text"])

In [ ]:
results['pred_keywords']

In [ ]:
rouge.compute(predictions=results["pred_keywords"], references=results["keywords"], rouge_types=["rouge2"])["rouge2"].mid

